In [1]:
import pandas as pd
from sklearn.cluster import KMeans

In [2]:
transactions = pd.read_csv('data/purchase/transactions.csv', nrows= 4e5) #5e7)
transactions = transactions[['id', 'dept', 'category']]

In [3]:
item_groups = transactions.groupby(['dept', 'category'])
id_groups = transactions.groupby('id')
items = item_groups.groups.keys()

In [4]:
def binarize(df):
    vals = pd.Series(0, index = pd.Index(items, tupleize_cols = False))
    for row in df.itertuples(index = False):
        vals[row] = 1
    return vals
df = id_groups.apply(binarize)

In [5]:
purchase_2 = KMeans(2, random_state = 0).fit_predict(df)
purchase_10 = KMeans(10, random_state = 0).fit_predict(df)
purchase_20 = KMeans(20, random_state = 0).fit_predict(df)
purchase_50 = KMeans(50, random_state = 0).fit_predict(df)
purchase_100 = KMeans(100, random_state = 0).fit_predict(df)